# Homework 7 - ME 364 (Spring 2022)

Use the BIKED dataset for the last homework assignment, also attached to this notebook for this in class assignment. Create a new dataset that only includes bike classes ROAD, MTB, TRACK, OTHER, DIRT JUMP, TOURING, CYCLOCROSS, and POLO.

## <font color='red'>__Question__</font>:

Use the methods we covered in class for feature selection and hyperparameter tuning to develop a Logistic Regression model and an SVM model. The goal here is to have each model perform better than the model you developed for homework 6. Compare the model outcome with what you got from the models you developed in homework 6. You don't need to re-create those models here. Just report their evaluation metrics from that notebook along with the evaluation metrics of your models developed here.

### Prepare Data

#### Import Data

In [2]:
import pandas as pd

url = 'https://raw.githubusercontent.com/yairg98/Data-Driven-Problem-Solving/main/Assignments/Homework%207/Biked_Dataset_Reduced.csv'
df = pd.read_csv(url)

df.head()

,SSSIDECX3,SSSIDECX2,SSSIDECX1,SSSIDECY2,SSSIDECY1,STEMBENDS,FRONTROTORBOLTS,Shoe up angle,Down tube front diameter,LRTHICK,...,Top tube type OHCLASS: 1,BRAZEonFDTYPE OHCLASS: FD9000F,BRAZEonFDTYPE OHCLASS: FD9070F,CSAUX3_MM_RATIO OHCLASS: 0,CSAUX3_MM_RATIO OHCLASS: 1,bottle SEATTUBE0 show OHCLASS: False,bottle SEATTUBE0 show OHCLASS: True,bottle DOWNTUBE0 show OHCLASS: False,bottle DOWNTUBE0 show OHCLASS: True,Bicycle_Class
0,0.115968,0.213580,0.240722,0.173913,0.295455,0,0.75,0.906475,0.323077,0,...,1,1,0,1,0,1,0,1,0,ROAD
1,0.115968,0.171084,0.199779,0.173913,0.295455,0,0.75,0.906475,0.293077,0,...,0,1,0,1,0,1,0,1,0,DIRT_JUMP
2,0.115968,0.267053,0.292434,0.173913,0.295455,0,0.75,0.899281,0.246154,0,...,0,1,0,1,0,1,0,1,0,POLO
3,0.115968,0.215305,0.242409,0.173913,0.295455,0,0.75,0.899281,0.246154,0,...,0,1,0,1,0,1,0,1,0,ROAD
4,0.115968,0.233025,0.259668,0.173913,0.295455,0,0.75,0.906475,0.293077,0,...,0,1,0,1,0,1,0,1,0,DIRT_JUMP


#### Filter Data

In [ ]:
# Limit data to list of included Bicycle_Class values from HW6

included = ['ROAD', 'MTB', 'TRACK', 'OTHER', 'DIRT_JUMP', 'TOURING', 'CYCLOCROSS', 'POLO']
df = df[df['Bicycle_Class'].isin(included)]

In [ ]:
# Limit data to target (Bicycle_Class) and 50 most predictive features

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np

model = LogisticRegression ( multi_class='multinomial' )
X = np.array(df.iloc[:,:-1])
MinMaxscaler = MinMaxScaler( ) # define min-max scaler
X_in = MinMaxscaler.fit_transform(X) # transform data
y = np.array(df['Bicycle_Class'] )

# fit the model
model.fit(X_in , y)

# get importance
importance = model.coef_[0]

# Top 50 important features
import numpy as np
importanceABS = np.abs( importance )
importanceABS.argsort()[-50:] [: :-1]

# Limit to only top 50 important features
top_50 = df.columns[importanceABS.argsort( )[-50: ][::-1]]
df = df[list(top_50)+['Bicycle_Class']]

#### Normalize and Split Data

In [ ]:
from sklearn.model_selection import train_test_split

# Select x and y data for logistic regression classifier
x_data=np.array(df[top_50])
y_data=df['Bicycle_Class']

# Normalize the data
MinMaxscaler = MinMaxScaler()  # define min max scaler
x_data_scaled = MinMaxscaler.fit_transform(x_data)  # transform data

# Split data into training and testing sets
x_train,x_test,y_train,y_test=train_test_split(x_data_scaled,y_data,test_size=0.25)

### Create Logistic Regression Classifier